In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Getting the upcoming 10 events
2024-04-14T20:00:00-04:00 Meeting Subject
2024-04-20T11:00:00-04:00 Meeting


In [2]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
from datetime import datetime, timedelta, timezone
import pytz
import os


def meeting_invite_email(service, summary, start_time_str, duration, attendees):
    # Parse the start time string into a datetime object
    start_time = datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%S%z')
    
    # Convert it to 'America/New_York' timezone
    eastern = pytz.timezone('America/New_York')
    start_time = start_time.astimezone(eastern)
    
    # Calculate the end time based on the duration
    end_time = start_time + timedelta(hours=duration)
    
    # Prepare the event body with the proper timezone
    event_body = {
        'summary': summary,
        'start': {
            'dateTime': start_time.isoformat(),
            'timeZone': 'America/New_York'
        },
        'end': {
            'dateTime': end_time.isoformat(),
            'timeZone': 'America/New_York'
        },
        'attendees': [{'email': email} for email in attendees],
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }
    
    # Create and send the event invite
    event = service.events().insert(calendarId='primary', body=event_body).execute()
    print(f"Meeting invite sent: {event.get('htmlLink')}")






In [3]:
def main():
    # ... (Your existing code)

    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("calendar", "v3", credentials=creds)

        # ... (Your existing code)

    except HttpError as error:
        print(f"An error occurred: {error}")
        return

    summary = "Team Meeting"
    start_time = "2024-04-30T10:00:00-05:00"  # Example start time in string format with timezone
    duration = 1  # Duration in hours
    attendees = ["xinhewu88@gmail.com"]  # Replace with actual attendee emails

    meeting_invite_email(service, summary, start_time, duration, attendees)



if __name__ == "__main__":
    main()

Meeting invite sent: https://www.google.com/calendar/event?eid=dDIzYjdnOXN0cHZwZGNjcHRvNmUwZm04cGcgeGluaHd1QHVtaWNoLmVkdQ
